In [4]:
pip install openai==0.28

Note: you may need to restart the kernel to use updated packages.


In [11]:
#pip install pyaudio

  Obtaining dependency information for pyaudio from https://files.pythonhosted.org/packages/82/d8/f043c854aad450a76e476b0cf9cda1956419e1dacf1062eb9df3c0055abe/PyAudio-0.2.14-cp311-cp311-win_amd64.whl.metadata
   ---------------------------------------- 0.0/164.1 kB ? eta -:--:--
   ---------------------------------------- 0.0/164.1 kB ? eta -:--:--
   -- ------------------------------------- 10.2/164.1 kB ? eta -:--:--
   --------- ----------------------------- 41.0/164.1 kB 487.6 kB/s eta 0:00:01
   ----------------- --------------------- 71.7/164.1 kB 653.6 kB/s eta 0:00:01
   ----------------- --------------------- 71.7/164.1 kB 653.6 kB/s eta 0:00:01
   -------------------------------------- 164.1/164.1 kB 979.8 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [1]:
#pip install gtts

In [1]:
import openai
import librosa
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import sounddevice as sd
import io
import tempfile
import pyaudio
import wave
import os

In [2]:
openai.api_key=""

In [5]:
# File address stored in a variable
file_path = r"D:\dys\Module 2\Disarthria-20240315T142530Z-001\Disarthria\Backspace\F02_B1_C2_M4.wav"

In [6]:
with open(file_path, "rb") as audio_file:
    # Transcribe the audio
    transcript = openai.Audio.transcribe(
        file=audio_file,
        model="whisper-1",
        response_format="text",
        language="en"
    )

print(transcript)

Back...space...



In [7]:
# Define a function to record live voice using microphone
def record_audio(duration=5, output_file="temp.wav"):
    CHUNK = 1024
    FORMAT = pyaudio.paInt16
    CHANNELS = 1
    RATE = 44100

    p = pyaudio.PyAudio()

    stream = p.open(format=FORMAT,
                    channels=CHANNELS,
                    rate=RATE,
                    input=True,
                    frames_per_buffer=CHUNK)

    frames = []

    print("Recording...")

    for i in range(0, int(RATE / CHUNK * duration)):
        data = stream.read(CHUNK)
        frames.append(data)

    print("Finished recording.")

    stream.stop_stream()
    stream.close()
    p.terminate()

    # Save recorded audio to a WAV file
    wf = wave.open(output_file, 'wb')
    wf.setnchannels(CHANNELS)
    wf.setsampwidth(p.get_sample_size(FORMAT))
    wf.setframerate(RATE)
    wf.writeframes(b''.join(frames))
    wf.close()

# Define a function to transcribe audio using OpenAI API
def transcribe_audio(audio_file):
    with open(audio_file, "rb") as f:
        response = openai.Audio.transcribe(
            file=f,
            model="whisper-1",
            response_format="text",
            language="en"
        )
    return response

# Record audio for 5 seconds
recorded_file = "temp.wav"
record_audio(duration=10, output_file=recorded_file)

Recording...
Finished recording.


In [29]:
# Transcribe the recorded audio
transcription = transcribe_audio(recorded_file)
print("Transcription:", transcription)

# Remove the temporary audio file
os.remove(recorded_file)

AuthenticationError: Incorrect API key provided: sk-TYLPK***************************************wv45. You can find your API key at https://platform.openai.com/account/api-keys.

In [27]:
# Assuming you already have the 'transcript' variable containing the text
text_to_speak = transcript

# Provide the language (optional)
language = 'en'

NameError: name 'transcript' is not defined

In [2]:
# Function to extract features
def extract_features(audio_file):
    y, sr = librosa.load(audio_file, sr=None)
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
    delta = librosa.feature.delta(mfccs)
    delta_delta = librosa.feature.delta(mfccs, order=2)
    return mfccs, delta, delta_delta


In [3]:
# Example usage
dysarthric_audio_file = r'D:\dys\Dysarthria and Non Dysarthria\Dataset\Female_dysarthria\F01\Session1\Wav\0008.wav'
normal_audio_file = r'D:\dys\Dysarthria and Non Dysarthria\Dataset\Female_Non_Dysarthria\FC01\Session1\Wav\0030.wav'

# Extract features for dysarthric and normal voices
dysarthric_mfccs, dysarthric_delta, dysarthric_delta_delta = extract_features(dysarthric_audio_file)
normal_mfccs, normal_delta, normal_delta_delta = extract_features(normal_audio_file)


In [135]:
def dysarthria(text):
    messages = [
        {"role": "system", "content": """You are given the automatic speech 
recognition result of a speaker with dysarthria, 
which may contain errors due to inaccurate 
pronunciation. Remove the speech recognition 
errors and correct the sentence. Only respond with 
the corrected results, without mentioning the instruction."""},
        {"role": "user", "content": f"""Analyze the following text and make nearly correct sentence or word if sentence or word make sense dont change give same word: {text} """}
    ]

    response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages = messages,
    
    
    stop=None,
    temperature=0.3)

    response_text=response.choices[0].message.content.strip().lower()

    return response_text


In [136]:
input = transcript
response = dysarthria(transcript)
print(response)

okay, ma'am. my baby ate a cupcake on the train. okay, she's repeating. my baby ate a cupcake on the train. say it one more time. my baby ate a cupcake on the train. okay, can you say west register street? west register street. okay, say riding artillery brigade. riding artillery brigade. okay, tell me what the weather's like outside. the weather is very hot and humid. okay, all right. thank you very much.


In [10]:
from gtts import gTTS

In [11]:
# Assuming you already have the 'transcript' variable containing the text
text_to_speak = transcript

# Provide the language (optional)
language = 'en'



In [12]:
# Provide the file name for the generated audio file
output_file = 'output_audio.mp3'

# Initialize the gTTS object with the text and language
tts = gTTS(text=text_to_speak, lang=language, slow=False)

# Save the generated speech to a file
tts.save(output_file)

# Play the audio file
os.system(f'start {output_file}')

0